In [1]:
#此脚本用来寻找公告日之前的停牌日期
import tushare as ts
import pandas as pd

In [2]:
pro = ts.pro_api()

In [3]:
li = pd.read_excel('../data/databg.xlsx')

In [11]:
dftp = pro.suspend(ts_code='600179.SH', suspend_date='', resume_date='', fields='')

In [18]:
dftp['suspend_date'][10:20].values

array(['20180510', '20180509', '20180508', '20180507', '20180504',
       '20180503', '20180502', '20180427', '20180426', '20180425'],
      dtype=object)

In [13]:
from datetime import datetime 
a = datetime.strptime('20141201', '%Y%m%d')
b = datetime.strptime('20150513', '%Y%m%d')
a-b

datetime.timedelta(days=-163)

In [4]:
from datetime import datetime 
def get_max_nbt(ia):
    '这是一个验证函数防止低级统计错误'
    q1 = datetime.strptime(ia[0], '%Y%m%d')
    max = 0
    for i in ia[0:-1]:
        q2 = datetime.strptime(i, '%Y%m%d')
        if((q1 - q2).days > max):
            max = (q1 - q2).days
        q1 = q2
    return max


def gettprq(tscode,knowdate):
    fo1 = knowdate
    fo2 = '0'
    fo0 = '0'
    t1 = 0
    t2 = 0
    dftp = pro.suspend(ts_code=tscode, suspend_date='', resume_date='', fields='')
    for i in range(len(dftp)):
        if((dftp['resume_date'][i] != None) and (int(dftp['suspend_date'][i])) < int(knowdate)):
            if(fo1 == knowdate):
                fo1 = dftp['suspend_date'][i]
                if(i>1):
                    fo0 = dftp['suspend_date'][i-1]
                t1 = i
            else:
                fo2 = dftp['suspend_date'][i-1]
                t2 = i
                break
        else:
            continue
    fo3 = 0
    if(t2 != 0):
        fo3 = get_max_nbt(dftp['suspend_date'][t1:t2].values)
    return fo0,fo1,fo2,fo3


In [4]:
#寻找复牌日期早上起来写吧
def gettprq(tscode,knowdate):
    findout = knowdate
    findout2 = knowdate
    findout3 = '0'
    dftp = pro.suspend(ts_code=tscode, suspend_date='', resume_date='', fields='')
    for susdate in dftp['suspend_date']:
        finddate = int(susdate)    #循环中的停牌日期
        kndate = int(knowdate)     #公告日期
        if(finddate >= kndate):    #如果循环日期在公告日期之后，两个tprq返回此时的循环日期并继续
            findout = str(finddate)
            findout2 = str(finddate) #一直循环到两个tprq都到了公告日期
            continue
        else:      #如果循环日期在公告日期之前
            if((int(findout2)-finddate)>20): #如果循环日期之前的20天前都没有停牌过
                 findout3 = str(finddate)   #fo3是最早的停牌日期
                 break
            else:                   #如果这个停牌日期距离上一个停牌日期在20天以内，
                findout2 = str(finddate) 
                continue
    return findout,findout2,findout3

In [5]:
timelist1 = []
ii = 0
for i in li['ts_code']:
    d=li[li['ts_code']==i]['news_date1'][ii]  
    tetime = d.strftime('%Y%m%d')  # 公告日期
    ii = ii+1

    tprq0,tprq,tprq2,tprq3 = gettprq(i,tetime)
    if(int(tetime) == int(tprq)):
        print(ii)
    timelist1.append([tprq,tprq2,tprq3,tprq0])

14
25
39
44
46
50


In [6]:
li[['tingpai','tingpai2','tingpai3','tprrr0']] = timelist1

In [7]:
li.to_excel('../data/data_h2.xlsx')

In [9]:
aa = pro.suspend(ts_code='002161.SZ', suspend_date='', resume_date='', fields='')

In [ ]:
###怎么说呢，这一步很傻逼，我也不知道我在干什么，但是没有这么一步的话整个数据都是错误的，但是没有文献提及